In [1]:
from flask import Flask, render_template, jsonify, request, redirect, url_for, Response, session
import time
import random
import string
import logging
import logging.handlers
import sqlite3
import yfinance as yf
import plotly.offline as po
import plotly.graph_objs as go
import plotly.express as px
import json
import plotly
# template_dir = os.path.abspath('/templates')
app = Flask(__name__)
app.secret_key = 'your_secret_key'

log = logging.getLogger('flask')
log.setLevel('INFO')
h = logging.handlers.TimedRotatingFileHandler(
    'logs/nlog', when='midnight', backupCount=9999)
f = logging.Formatter('%(asctime)s %(levelname)-8s %(message)s')
h.setFormatter(f)
log.addHandler(h)


@app.route('/')
@app.route('/index/')
def index():
    title = 'Portfolio Tracker'
    return render_template('index.html', title=title)

@app.route('/register_page')
def register1():
    title = 'Register Page'
    return render_template('register1.html', title=title)

def CheckIfCorrectCredentials(loginName,password):

    conn = sqlite3.connect('DataBase1.db')

    c = conn.cursor()

    params=[loginName,password]
    c.execute("Select * from users where loginName==? and password==?",params)
    print("\nPresent all data in student table:")

    #Get all results,assign them to the list,fecthall() returns empty list if no results
    listOfResults=c.fetchall()

    # Save (commit) the changes
    conn.commit()

    # We close the connection and free all resources
    conn.close()
    
    # If given loginName and password exist in database then exactly one row should be returned.
    # We assume that loginNames are unique.
    if (len(listOfResults)==1):
        return True
    return False

@app.route('/login_page')
def login_page():
    title = 'Login Page'
    return render_template('login.html', title=title)
@app.route('/login/', methods=['POST','GET'])
def login():
    loginName = request.form.get('login_name', default=False)
    password = request.form.get('password', default=False)

    log.info(str(loginName) + ' ' + request.method + ' ' +
             request.url + ' ' + request.environ.get('REMOTE_ADDR'))

    if (CheckIfCorrectCredentials(loginName,password)==True):
        session['loginName'] = loginName            # stores the loginName of logged User  
        return redirect('/portfolioOverview')
    else:
        title = 'Login Page'
        return render_template('login.html', error = True, title=title)


@app.route('/register', methods=['POST','GET'])
def register():
    loginName = request.form.get('login_name', default=False)
    password1 = request.form.get('password1', default=False)
    password2 = request.form.get('password2', default=False)

    name = request.form.get('name', default=False)
    surname = request.form.get('surname', default=False)
    birthDate = request.form.get('birthDate', default=False)

    city = request.form.get('city', default=False)
    zipCode = request.form.get('zipCode', default=False)
    street = request.form.get('street', default=False)
    houseNo = request.form.get('houseNo', default=False)
    flatNo =request.form.get('flatNo', default=False)

    conn = sqlite3.connect('DataBase1.db')

    c = conn.cursor()

    c.execute("select userID from users order by userID desc limit 1")
    userID = c.fetchall()
    userID = userID[0][0]
    userID += 1
    
    c.execute("select personalInfoID from userPersonalInfo order by PersonalInfoID desc limit 1")
    personalInfoID = c.fetchall()
    personalInfoID = personalInfoID[0][0]
    personalInfoID += 1

    params1 = [loginName, password1]
    c.execute("insert into users(loginName, password) values (?, ?)", params1)

    params2 = [name, surname, birthDate, userID]
    c.execute("insert into userPersonalInfo(name, surname, birthDate, userID) values (?, ?, ?, ?)", params2)

    params3 = [city, zipCode, street, houseNo, flatNo, personalInfoID]
    c.execute("insert into userAddress(city, zipCode, street, houseNo, flatNo, personalInfoID) values (?, ?, ?, ?, ?, ?)", params3)

    conn.commit()
    conn.close()

    title = 'Portfolio Overview'

    return render_template('portfolioOverview.html', title=title)


def takeAllUserData():

    loginName = session['loginName']

    conn = sqlite3.connect('DataBase1.db')
    c = conn.cursor()
    c.execute("SELECT userID FROM users WHERE loginName = ?", (loginName,))
    
    userID = c.fetchone()[0]
    
    c.execute("select * from userPersonalInfo where userID = ?", (userID,))

    userPersonalInfo = c.fetchall()[0]
    personalInfoID = userPersonalInfo[0]

    c.execute("select * from userAddress where personalInfoID = ?", (personalInfoID,))
    userAddress = c.fetchall()[0]
    
    conn.commit()
    conn.close()
    
    name = userPersonalInfo[1]
    surname = userPersonalInfo[2]
    birthDate = userPersonalInfo[3]

    city = userAddress[1]
    zipCode = userAddress[2]
    street = userAddress[3]
    houseNo = userAddress[4]
    flatNo = userAddress[5]

    return loginName, name, surname, birthDate, city, zipCode, street, houseNo, flatNo


@app.route('/stockMarketInfo')
def stockMarketInfo():
    title = 'Stock Market Info'
    return render_template('stockMarketInfo.html', title=title)

@app.route('/portfolioEditor')
def portfolioEditor():
    title = 'Portfolio Editor'
    return render_template('portfolioEditor.html', title=title)

@app.route('/personalData')
def personalData():
    data = takeAllUserData()        #  argument after ** must be a mapping, not tuple"
    data_dict = {
        'loginName': data[0],
        'name': data[1],
        'surname': data[2],
        'birthDate': data[3],
        'city': data[4],
        'zipCode': data[5],
        'street': data[6],
        'houseNo': data[7],
        'flatNo': data[8]
    }
    title = 'Personal Data'
    return render_template('personalData.html', title=title, **data_dict)

# function to show teh page without reloading
@app.route('/changePersonalDataPage')
def get_data():
    title = "Change Personal Data"
    return render_template('changePersonalData.html', title=title)


@app.route('/changePersonalData', methods=['POST','GET'])
def changePersonalData():
    newLoginName = request.form.get('new_login_name', default=False)

    newName = request.form.get('new_name', default=False)
    newSurname = request.form.get('new_surname', default=False)
    newBirthDate = request.form.get('new_birthDate', default=False)

    newCity = request.form.get('new_city', default=False)
    newZipCode = request.form.get('new_zipCode', default=False)
    newStreet = request.form.get('new_street', default=False)
    newHouseNo = request.form.get('new_houseNo', default=False)
    newFlatNo =request.form.get('new_flatNo', default=False)

    conn = sqlite3.connect('DataBase1.db')

    c = conn.cursor()

    loginName = session['loginName']
    

    c.execute("SELECT userID FROM users WHERE loginName = ?", (loginName,))
    userID = c.fetchone()[0]

    c.execute("select personalInfoID from userPersonalInfo where userID = ?", (userID,))
    personalInfoID = c.fetchone()[0]

    c.execute("update users set loginName = ? where userID = ?", (newLoginName, userID))

    c.execute("update userPersonalInfo set name = ?, surname = ?, birthDate = ? where userID = ?", (newName, newSurname, newBirthDate, userID))

    c.execute("update userAddress set city = ?, zipCode = ?, street = ?, houseNo = ?, flatNo = ? where personalInfoID = ?", (newCity, newZipCode, newStreet, newHouseNo, newFlatNo, personalInfoID))

    conn.commit()
    conn.close()

    title = 'Portfolio Overview'

    return render_template('personalData.html', title=title)

@app.route('/callback/<endpoint>')
def cb(endpoint):   
    if endpoint == "getStock":
        return gm(request.args.get('data'),request.args.get('period'),request.args.get('interval'))
    elif endpoint == "getInfo":
        stock = request.args.get('data')
        st = yf.Ticker(stock)
        return json.dumps(st.info)
    else:
        return "Bad endpoint", 400

# Return the JSON data for the Plotly graph
def gm(stock,period, interval):
    st = yf.Ticker(stock)
  
    # Create a line graph
    df = st.history(period=(period), interval=interval)
    df=df.reset_index()
    df.columns = ['Date-Time']+list(df.columns[1:])
    max = (df['Open'].max())
    min = (df['Open'].min())
    range = max - min
    margin = range * 0.05
    max = max + margin
    min = min - margin
    print(df)
    fig = px.area(df, x='Date-Time', y="Open",
        hover_data=("Open","Close","Volume"), 
        range_y=(min,max), template="seaborn" )


    # Create a JSON representation of the graph
    graphJSON = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
    return graphJSON



@app.route('/changePassword')
def changePassword():
    title = 'Change Password'
    return render_template('changePassword.html', title=title)

@app.route('/changePasswordFun', methods=['POST','GET'])
def changePasswordFun():
    password = request.form.get('password', default=False)
    new_password = request.form.get('new_password1', default=False)

    loginName = session['loginName']

    if CheckIfCorrectCredentials(loginName, password) == True:
        changePasswordDB(loginName, new_password)
        title = 'changePassword'
        return render_template('changePassword.html', success = True, title=title)
    else:
        title = 'changePassword'
        return render_template('changePassword.html', error = True, title=title)
    

def changePasswordDB(loginName, new_password):
    conn = sqlite3.connect('DataBase1.db')
    c = conn.cursor()

    # ewentualnie dopisać u samej góry funkcję na pobieranie ID i przechowywać go w session, żeby nie uderzać do bazy za każdym razem

    c.execute("update users set password = ? where loginName = ?", (new_password, loginName))

    conn.commit()
    conn.close()



@app.route('/portfolioOverview')
def portfolioOverview():
    title = 'Portfolio Overview'
    return render_template('portfolioOverview.html', title=title)

  


app.run(host='localhost', port=63700, use_reloader=False, debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:63700
Press CTRL+C to quit
127.0.0.1 - - [19/Jan/2023 22:57:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2023 22:57:05] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [19/Jan/2023 22:57:07] "GET /login_page HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2023 22:57:07] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [19/Jan/2023 22:57:10] "POST /login HTTP/1.1" 308 -
127.0.0.1 - - [19/Jan/2023 22:57:11] "POST /login/ HTTP/1.1" 302 -



Present all data in student table:


127.0.0.1 - - [19/Jan/2023 22:57:11] "GET /portfolioOverview HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2023 22:57:11] "GET /static/css/style.css HTTP/1.1" 304 -


                    Date-Time        Open        High         Low       Close  \
0   2023-01-19 09:30:00-05:00  233.779999  234.979996  233.779999  234.899994   
1   2023-01-19 09:31:00-05:00  234.880005  234.979996  234.327896  234.360001   
2   2023-01-19 09:32:00-05:00  234.369995  235.520004  234.259995  235.029999   
3   2023-01-19 09:33:00-05:00  235.029999  235.219894  234.770004  235.130005   
4   2023-01-19 09:34:00-05:00  235.110001  235.139999  234.630005  234.729996   
..                        ...         ...         ...         ...         ...   
386 2023-01-19 15:56:00-05:00  232.029999  232.139999  231.979996  232.050003   
387 2023-01-19 15:57:00-05:00  232.050003  232.070007  231.970001  232.035004   
388 2023-01-19 15:58:00-05:00  232.039993  232.139999  231.979996  232.009995   
389 2023-01-19 15:59:00-05:00  232.020004  232.059998  231.580002  232.029999   
390 2023-01-19 16:00:00-05:00  231.929993  231.929993  231.929993  231.929993   

     Volume  Dividends  Sto

...

127.0.0.1 - - [19/Jan/2023 22:57:31] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [19/Jan/2023 22:57:31] "GET /script.js HTTP/1.1" 404 -
127.0.0.1 - - [19/Jan/2023 22:57:31] "GET /static/script.js HTTP/1.1" 304 -
127.0.0.1 - - [19/Jan/2023 22:57:50] "POST /changePasswordFun HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2023 22:57:50] "GET /static/css/style.css HTTP/1.1" 304 -



Present all data in student table:


127.0.0.1 - - [19/Jan/2023 22:57:50] "GET /script.js HTTP/1.1" 404 -
127.0.0.1 - - [19/Jan/2023 22:57:50] "GET /static/script.js HTTP/1.1" 304 -
127.0.0.1 - - [19/Jan/2023 22:57:56] "POST /changePasswordFun HTTP/1.1" 200 -



Present all data in student table:


127.0.0.1 - - [19/Jan/2023 22:57:56] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [19/Jan/2023 22:57:56] "GET /script.js HTTP/1.1" 404 -
127.0.0.1 - - [19/Jan/2023 22:57:56] "GET /static/script.js HTTP/1.1" 304 -


In [1]:
import sqlite3

def changePasswordDB(loginName, new_password):
    conn = sqlite3.connect('DataBase1.db')
    c = conn.cursor()

    # ewentualnie dopisać u samej góry funkcję na pobieranie ID i przechowywać go w session, żeby nie uderzać do bazy za każdym razem

    c.execute("update users set password = ? where loginName = ?", (new_password, loginName))

    conn.commit()
    conn.close()


changePasswordDB("admin", "123")

In [3]:
conn = sqlite3.connect('DataBase1.db')
c = conn.cursor()
c.execute("SELECT assetName FROM assets")
symbols = [row[0] for row in c.fetchall()]
print(symbols)
conn.close()

symbol = 'AAPL'
start = '2020-01-01'
end = '2020-12-31'
stock = yf.Ticker(symbol)
df = stock.history(start=start, end=end)
print(df)
data = {
    'labels': df.index.tolist(),
    'datasets': [{
        'label': symbol,
        'data': df[['Open', 'High', 'Low', 'Close']].values.tolist(),
        'type': 'candlestick',
    }],
    'symbols': symbols
}

print(data)





@app.route("/get_asset_names")
def get_asset_names():
    conn = sqlite3.connect('DataBase1.db')
    c = conn.cursor()
    c.execute("SELECT assetName FROM assets")
    asset_names = [row[0] for row in c.fetchall()]
    conn.close()
    return jsonify(asset_names)

@app.route("/get_data", methods=["POST"])
def get_data():
    asset_id = request.form["asset_id"]
    start_date = request.form["start_date"]
    end_date = request.form["end_date"]
    stock = yf.Ticker(asset_id)
    df = stock.history(start=start_date, end=end_date)
    print(df)
    data = {
        'labels': df.index.tolist(),
        'datasets': [{
            'label': asset_id,
            'data': df[['Open', 'High', 'Low', 'Close']].values.tolist(),
            'type': 'candlestick',
        }]
    }
    
    trace = go.Candlestick(x=df.index,open=df['Open'],high=df['High'],low=df['Low'],close=df['Close'],
                            name = 'asset_id')
    data = [trace]
    layout = {'title': sid}
    fig = dict(data=data, layout=layout)
    plot_1 = po.plot(fig, filename='templates/stock.html',auto_open=False)
    
    return jsonify(plot_1)


@app.route('/portfolioOverview', methods=['POST','GET'])
def stock_data():
    conn = sqlite3.connect('DataBase1.db')
    c = conn.cursor()
    c.execute("SELECT assetName FROM assets")
    symbols = [row[0] for row in c.fetchall()]
    print(symbols)
    conn.close()

    symbol = request.args.get('symbol')
    start = request.args.get('start')
    end = request.args.get('end')
    print(symbol)

    stock = yf.Ticker(symbol)
    df = stock.history(start=start, end=end)
    data = {
        'labels': df.index.tolist(),
        'datasets': [{
            'label': symbol,
            'data': df[['Open', 'High', 'Low', 'Close']].values.tolist(),
            'type': 'candlestick',
        }],
        'symbols': symbols
    }

    return render_template('portfolioOverview.html', symbols=symbols, data=data)

['AAPL']
                                 Open        High         Low       Close  \
Date                                                                        
2020-01-02 00:00:00-05:00   72.554909   73.622761   72.297746   73.561531   
2020-01-03 00:00:00-05:00   72.777797   73.617869   72.618601   72.846375   
2020-01-06 00:00:00-05:00   71.954866   73.466014   71.700148   73.426826   
2020-01-07 00:00:00-05:00   73.436626   73.696240   72.858620   73.081497   
2020-01-08 00:00:00-05:00   72.780239   74.563252   72.780239   74.257103   
...                               ...         ...         ...         ...   
2020-12-23 00:00:00-05:00  130.592318  130.859104  129.228683  129.406555   
2020-12-24 00:00:00-05:00  129.762303  131.876918  129.544911  130.404587   
2020-12-28 00:00:00-05:00  132.400619  135.710872  131.926302  135.068588   
2020-12-29 00:00:00-05:00  136.412459  137.143671  132.746460  133.270172   
2020-12-30 00:00:00-05:00  133.971757  134.376897  131.817608  132.